In [2]:
import numpy as np
import sys
import tensorflow as tf
from keras.losses import Loss
from sklearn.preprocessing import StandardScaler
from scipy.sparse import csr_matrix
import pandas as pd
import datetime
import time
import io
from sklearn.model_selection import train_test_split
import scipy.io as sio

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
def load_data(dataset, kappa, exp_id, original_X=False, extra_str=""):
    path = '/content/drive/MyDrive/Causal_network_matching/data/'+dataset+str(kappa)+'/'+str(dataset)+''+str(exp_id)+'.mat'
    print(path)
    data = sio.loadmat(path)
    A = data['Network']  # csr matrix

    if not original_X:
        X = data['X_100']
    else:
        X = data['Attributes']

    mu_1 = data['Y1']
    mu_0 = data['Y0']
    T = data['T']

    T = T.flatten()
    mu_1 = mu_1.flatten()
    mu_0 = mu_0.flatten()

    Y_observed = []
    for i in range(len(T)):
        if T[i] == 1:
            Y_observed.append(mu_1[i])
        else:
            Y_observed.append(mu_0[i])

    Y_observed = np.array(Y_observed)
    X = X.todense()


    X_train, X_test, Y_factual_train, _, T_train, _, mu_0_train, mu_0_test, mu_1_train, mu_1_test = train_test_split(X, Y_observed, T, mu_0, mu_1, test_size=0.2)

    X_train = X_train.astype('float32')
    Y_factual_train = Y_factual_train.astype('float32') #most GPUs only compute 32-bit floats
    T_train = T_train.astype('float32')
    mu_0_train = mu_0_train.astype('float32')
    mu_1_train = mu_1_train.astype('float32')

    X_test = X_test.astype('float32')
    mu_0_test = mu_0_test.astype('float32')
    mu_1_test = mu_1_test.astype('float32')

    data_train={'x':X_train,'y':Y_factual_train,'t':T_train,'mu_0':mu_0_train,'mu_1':mu_1_train}
    data_train['t']=data_train['t'].reshape(-1,1) #we're just padding one dimensional vectors with an additional dimension
    data_train['y']=data_train['y'].reshape(-1,1)
    data_train['mu_0']=data_train['mu_0'].reshape(-1,1) #we're just padding one dimensional vectors with an additional dimension
    data_train['mu_1']=data_train['mu_1'].reshape(-1,1)

    #rescaling y between 0 and 1 often makes training of DL regressors easier
    data_train['y_scaler'] = StandardScaler().fit(data_train['y'])
    data_train['ys'] = data_train['y_scaler'].transform(data_train['y'])

    data_test={'x':X_test,'mu_0':mu_0_test,'mu_1':mu_1_test}
    data_test['y_scaler'] = data_train['y_scaler']
    data_test['mu_0']=data_test['mu_0'].reshape(-1,1) #we're just padding one dimensional vectors with an additional dimension
    data_test['mu_1']=data_test['mu_1'].reshape(-1,1)

    return data_train, data_test

In [5]:
from tensorflow.keras.layers import Layer
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Concatenate
from tensorflow.keras import regularizers
from tensorflow.keras import Model
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.metrics import binary_accuracy
from tensorflow.keras.losses import Loss
class EpsilonLayer(Layer):

    def __init__(self):
        super(EpsilonLayer, self).__init__()

    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        self.epsilon = self.add_weight(name='epsilon',
                                       shape=[1, 1],
                                       initializer='RandomNormal',
                                       #  initializer='ones',
                                       trainable=True)
        super(EpsilonLayer, self).build(input_shape)  # Be sure to call this at the end

    def call(self, inputs, **kwargs):
        #note there is only one epsilon were just duplicating it for conformability
        return self.epsilon * tf.ones_like(inputs)[:, 0:1]

def make_dragonnet(input_dim, reg_l2):

    x = Input(shape=(input_dim,), name='input')
    # representation
    phi = Dense(units=200, activation='elu', kernel_initializer='RandomNormal',name='phi_1')(x)
    phi = Dense(units=200, activation='elu', kernel_initializer='RandomNormal',name='phi_2')(phi)

    # HYPOTHESIS
    y0_hidden = Dense(units=100, activation='elu', kernel_regularizer=regularizers.l2(reg_l2),name='y0_hidden_1')(phi)
    y1_hidden = Dense(units=100, activation='elu', kernel_regularizer=regularizers.l2(reg_l2),name='y1_hidden_1')(phi)

    # second layer
    y0_hidden = Dense(units=100, activation='elu', kernel_regularizer=regularizers.l2(reg_l2),name='y0_hidden_2')(y0_hidden)
    y1_hidden = Dense(units=100, activation='elu', kernel_regularizer=regularizers.l2(reg_l2),name='y1_hidden_2')(y1_hidden)

    # third
    y0_predictions = Dense(units=1, activation=None, kernel_regularizer=regularizers.l2(reg_l2), name='y0_predictions')(y0_hidden)
    y1_predictions = Dense(units=1, activation=None, kernel_regularizer=regularizers.l2(reg_l2), name='y1_predictions')(y1_hidden)

    #propensity prediction
    #Note that the activation is actually sigmoid, but we will squish it in the loss function for numerical stability reasons
    t_predictions = Dense(units=1,activation=None,name='t_prediction')(phi)
    #Although the epsilon layer takes an input, it really just houses a free parameter.
    epsilons = EpsilonLayer()(t_predictions)
    concat_pred = Concatenate(1)([y0_predictions, y1_predictions,t_predictions,epsilons,phi])
    model = Model(inputs=x, outputs=concat_pred)
    return model


In [6]:
class Base_Loss(Loss):
    #initialize instance attributes
    def __init__(self, alpha=1.0):
        super().__init__()
        self.alpha = alpha
        self.name='standard_loss'

    def split_pred(self,concat_pred):
        #generic helper to make sure we dont make mistakes
        preds={}
        preds['y0_pred'] = concat_pred[:, 0]
        preds['y1_pred'] = concat_pred[:, 1]
        preds['t_pred'] = concat_pred[:, 2]
        preds['phi'] = concat_pred[:, 3:]
        return preds

    #for logging purposes only
    def treatment_acc(self,concat_true,concat_pred):
        t_true = concat_true[:, 1]
        p = self.split_pred(concat_pred)
        #Since this isn't used as a loss, I've used tf.reduce_mean for interpretability
        return tf.reduce_mean(binary_accuracy(t_true, tf.math.sigmoid(p['t_pred']), threshold=0.5))

    def treatment_bce(self,concat_true,concat_pred):
        t_true = concat_true[:, 1]
        p = self.split_pred(concat_pred)
        lossP = tf.reduce_sum(binary_crossentropy(t_true,p['t_pred'],from_logits=True))
        return lossP

    def regression_loss(self,concat_true,concat_pred):
        y_true = concat_true[:, 0]
        t_true = concat_true[:, 1]
        p = self.split_pred(concat_pred)
        loss0 = tf.reduce_sum((1. - t_true) * tf.square(y_true - p['y0_pred']))
        loss1 = tf.reduce_sum(t_true * tf.square(y_true - p['y1_pred']))
        return loss0+loss1

    def standard_loss(self,concat_true,concat_pred):
        lossR = self.regression_loss(concat_true,concat_pred)
        lossP = self.treatment_bce(concat_true,concat_pred)
        return lossR + self.alpha * lossP

    #compute loss
    def call(self, concat_true, concat_pred):
        return self.standard_loss(concat_true,concat_pred)

In [7]:
class TarReg_Loss(Base_Loss):
    #initialize instance attributes
    def __init__(self, alpha=1,beta=1):
        super().__init__()
        self.alpha = alpha
        self.beta=beta
        self.name='tarreg_loss'

    def split_pred(self,concat_pred):
        #generic helper to make sure we dont make mistakes
        preds={}
        preds['y0_pred'] = concat_pred[:, 0]
        preds['y1_pred'] = concat_pred[:, 1]
        preds['t_pred'] = concat_pred[:, 2]
        preds['epsilon'] = concat_pred[:, 3] #we're moving epsilon into slot three
        preds['phi'] = concat_pred[:, 4:]
        return preds

    def calc_hstar(self,concat_true,concat_pred):
        #step 2 above
        p=self.split_pred(concat_pred)
        y_true = concat_true[:, 0]
        t_true = concat_true[:, 1]

        t_pred = tf.math.sigmoid(concat_pred[:, 2])
        t_pred = (t_pred + 0.001) / 1.002 # a little numerical stability trick implemented by Shi
        y_pred = t_true * p['y1_pred'] + (1 - t_true) * p['y0_pred']

        #calling it cc for "clever covariate" as in SuperLearner TMLE literature
        cc = t_true / t_pred - (1 - t_true) / (1 - t_pred)
        h_star = y_pred + p['epsilon'] * cc
        return h_star

    def call(self,concat_true,concat_pred):
        y_true = concat_true[:, 0]

        standard_loss=self.standard_loss(concat_true,concat_pred)
        h_star=self.calc_hstar(concat_true,concat_pred)
        #step 3 above
        targeted_regularization = tf.reduce_sum(tf.square(y_true - h_star))

        # final
        loss = standard_loss + self.beta * targeted_regularization
        return loss

In [8]:

#https://towardsdatascience.com/implementing-macro-f1-score-in-keras-what-not-to-do-e9f1aa04029d
class Eval_metrics_train():
    def __init__(self,data):
        self.data=data #feed the callback the full dataset
        #needed for PEHEnn; Called in self.find_ynn
        self.data['o_idx']=tf.range(self.data['t'].shape[0])
        self.data['c_idx']=self.data['o_idx'][self.data['t'].squeeze()==0] #These are the indices of the control units
        self.data['t_idx']=self.data['o_idx'][self.data['t'].squeeze()==1] #These are the indices of the treated units

    def split_pred(self,concat_pred):
        preds={}
        preds['y0_pred'] = self.data['y_scaler'].inverse_transform(concat_pred[:, 0].reshape(-1, 1))
        preds['y1_pred'] = self.data['y_scaler'].inverse_transform(concat_pred[:, 1].reshape(-1, 1))
        preds['t_pred'] = concat_pred[:, 2]
        preds['epsilon'] = concat_pred[:, 3]
        preds['phi'] = concat_pred[:, 4:]
        return preds

    def ATE_absolute_error(self,concat_pred):
        p = self.split_pred(concat_pred)
        ATT_pred = tf.gather(params=self.data['y'], indices=self.data['t_idx']) - tf.gather(params=p['y0_pred'], indices=self.data['t_idx'])
        ATU_pred = tf.gather(params=p['y1_pred'], indices=self.data['c_idx']) - tf.gather(params=self.data['y'], indices=self.data['c_idx'])
        ATE_pred = tf.reduce_mean(tf.concat([ATT_pred,ATU_pred], axis=0)) #stitch em back up!
        ATE_actual = tf.reduce_mean(self.data['mu_1']-self.data['mu_0'])
        return tf.abs(ATE_actual- ATE_pred)

    def ITE_RMSE_error(self,concat_pred):
        #simulation only
        p = self.split_pred(concat_pred)
        y_1_treated_group = tf.gather(params=self.data['y'], indices=self.data['t_idx'])
        y_0_treated_group = tf.gather(params=p['y0_pred'], indices=self.data['t_idx'])

        mu_1_treated_group = tf.gather(params=self.data['mu_1'], indices=self.data['t_idx'])
        mu_0_treated_group = tf.gather(params=self.data['mu_0'], indices=self.data['t_idx'])


        treat_grp_error = (y_1_treated_group - y_0_treated_group) - (mu_1_treated_group - mu_0_treated_group)

        y_1_control_group = tf.gather(params=p['y1_pred'], indices=self.data['c_idx'])
        y_0_control_group = tf.gather(params=self.data['y'], indices=self.data['c_idx'])

        mu_1_control_group = tf.gather(params=self.data['mu_1'], indices=self.data['c_idx'])
        mu_0_control_group = tf.gather(params=self.data['mu_0'], indices=self.data['c_idx'])

        control_grp_error = (y_1_control_group - y_0_control_group) - (mu_1_control_group - mu_0_control_group)


        ITE_error = tf.concat([treat_grp_error, control_grp_error], axis=0)
        ITE_RMSE_error = tf.sqrt(tf.reduce_mean(tf.square(ITE_error)))
        return ITE_RMSE_error

    def compute_hstar(self,y0_pred,y1_pred,t_pred,t_true,epsilons):
        #helper for calculating the targeted regularization cate
        y_pred = t_true * y1_pred + (1 - t_true) * y0_pred
        cc = t_true / t_pred - (1 - t_true) / (1 - t_pred)
        h_star = y_pred + epsilons * cc
        return h_star

    def TARREG_ATE_absolute_error(self,concat_pred):
        #Final calculation of Targeted Regularization loss
        p = self.split_pred(concat_pred)
        t_pred = tf.math.sigmoid(p['t_pred'])
        t_pred = (t_pred + 0.001) / 1.002 # a little numerical stability trick implemented by Shi
        hstar_0=self.compute_hstar(p['y0_pred'],p['y1_pred'],t_pred,tf.zeros_like(p['epsilon']),p['epsilon'])
        hstar_1=self.compute_hstar(p['y0_pred'],p['y1_pred'],t_pred,tf.ones_like(p['epsilon']),p['epsilon'])
        ate_true=tf.reduce_mean(self.data['mu_1']-self.data['mu_0'])
        ate_tarreg_pred = tf.reduce_mean(hstar_1-hstar_0)
        return tf.abs(ate_true- ate_tarreg_pred)

In [9]:

#https://towardsdatascience.com/implementing-macro-f1-score-in-keras-what-not-to-do-e9f1aa04029d
class Eval_metrics_test():
    def __init__(self,data):
        self.data=data #feed the callback the full dataset

    def split_pred(self,concat_pred):
        preds={}
        preds['y0_pred'] = self.data['y_scaler'].inverse_transform(concat_pred[:, 0].reshape(-1, 1))
        preds['y1_pred'] = self.data['y_scaler'].inverse_transform(concat_pred[:, 1].reshape(-1, 1))
        preds['t_pred'] = concat_pred[:, 2]
        preds['epsilon'] = concat_pred[:, 3]
        preds['phi'] = concat_pred[:, 4:]
        return preds


    def PEHE(self,concat_pred):
        #simulation only
        p = self.split_pred(concat_pred)
        cate_err=tf.reduce_mean( tf.square( ( (self.data['mu_1']-self.data['mu_0']) - (p['y1_pred']-p['y0_pred']) ) ) )
        return tf.sqrt(cate_err)

    def PEHE_tareg(self,concat_pred):
        #simulation only
        p = self.split_pred(concat_pred)
        t_pred = tf.math.sigmoid(p['t_pred'])
        t_pred = (t_pred + 0.001) / 1.002 # a little numerical stability trick implemented by Shi
        hstar_0=self.compute_hstar(p['y0_pred'],p['y1_pred'],t_pred,tf.zeros_like(p['epsilon']),p['epsilon'])
        hstar_1=self.compute_hstar(p['y0_pred'],p['y1_pred'],t_pred,tf.ones_like(p['epsilon']),p['epsilon'])

        cate_err=tf.reduce_mean( tf.square( ( (self.data['mu_1']-self.data['mu_0']) - (hstar_1-hstar_0) ) ) )
        return tf.sqrt(cate_err)


    def compute_hstar(self,y0_pred,y1_pred,t_pred,t_true,epsilons):
        #helper for calculating the targeted regularization cate
        y_pred = t_true * y1_pred + (1 - t_true) * y0_pred
        cc = t_true / t_pred - (1 - t_true) / (1 - t_pred)
        h_star = y_pred + epsilons * cc
        return h_star

    def TARREG_ATE_absolute_error(self,concat_pred):
        #Final calculation of Targeted Regularization loss
        p = self.split_pred(concat_pred)
        t_pred = tf.math.sigmoid(p['t_pred'])
        t_pred = (t_pred + 0.001) / 1.002 # a little numerical stability trick implemented by Shi
        hstar_0=self.compute_hstar(p['y0_pred'],p['y1_pred'],t_pred,tf.zeros_like(p['epsilon']),p['epsilon'])
        hstar_1=self.compute_hstar(p['y0_pred'],p['y1_pred'],t_pred,tf.ones_like(p['epsilon']),p['epsilon'])
        ate_true=tf.reduce_mean(self.data['mu_1']-self.data['mu_0'])
        print(tf.shape(hstar_1-hstar_0))
        ate_tarreg_pred = tf.reduce_mean(hstar_1-hstar_0)
        return tf.abs(ate_true- ate_tarreg_pred)

In [19]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau, TerminateOnNaN
from tensorflow.keras.optimizers import SGD, Adam
import io
from IPython.display import display, clear_output
#Colab command to allow us to run Colab in TF2
%load_ext tensorboard
# data=get_news_data(1)
val_split=0.20
batch_size=4000
verbose=1
i = 0
tf.random.set_seed(i)
np.random.seed(i)
!rm -rf ./logs_dragonnet/
sim_evals = []
for j in range(0,10):
  clear_output(wait=True)
  print(j+1)
  data_train, data_test = load_data(dataset="BlogCatalog", kappa=2, exp_id=j)
  yt = np.concatenate([data_train['ys'], data_train['t']], 1)

  # Clear any logs from previous runs

  start_time = time.time()

  log_dir = "logs_dragonnet/fit/" +str(j)+"/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  file_writer = tf.summary.create_file_writer(log_dir + "/metrics")
  file_writer.set_as_default()
  tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=0)

  print(data_train['x'])


  sgd_callbacks = [
        TerminateOnNaN(),
        EarlyStopping(monitor='val_loss', patience=40, min_delta=0.),
        ReduceLROnPlateau(monitor='loss', factor=0.5, patience=5, verbose=verbose, mode='auto',
                          min_delta=0., cooldown=0, min_lr=0),
        tensorboard_callback]

  # sgd_lr = 0.00001 # Flickr
  sgd_lr = 0.00000006 # BlogCat
  momentum = 0.9

  dragonnet_model=make_dragonnet(data_train['x'].shape[1],.01)
  tarreg_loss=TarReg_Loss(alpha=1)

  dragonnet_model.compile(optimizer=SGD(learning_rate=sgd_lr, momentum=momentum, nesterov=True),
                      loss=tarreg_loss,
                 metrics=[tarreg_loss,tarreg_loss.regression_loss,tarreg_loss.treatment_acc])

  dragonnet_model.fit(x=data_train['x'],y=yt,
                 callbacks=sgd_callbacks,
                  validation_split=val_split,
                  epochs=100000,
                  batch_size=batch_size,
                  verbose=verbose)
  end_time = time.time()

  elapsed_time_seconds = end_time - start_time
  # Convert elapsed time to minutes
  elapsed_time_minutes = elapsed_time_seconds / 60

  Evaluation_metrics_insample = Eval_metrics_train(data_train)
  concat_pred_insample = dragonnet_model.predict(data_train['x'])
  ATE_abs_insample = Evaluation_metrics_insample.ATE_absolute_error(concat_pred_insample)
  ATE_TARREG_insample = Evaluation_metrics_insample.TARREG_ATE_absolute_error(concat_pred_insample)
  ITE_RMSE_insample = Evaluation_metrics_insample.ITE_RMSE_error(concat_pred_insample)

  Evaluation_metrics_outsample = Eval_metrics_test(data_test)
  concat_pred_outsample = dragonnet_model.predict(data_test['x'])
  ATE_TARREG_outsample = Evaluation_metrics_outsample.TARREG_ATE_absolute_error(concat_pred_outsample)
  PEHE_outsample = Evaluation_metrics_outsample.PEHE(concat_pred_outsample)
  PEHE_tareg_outsample = Evaluation_metrics_outsample.PEHE_tareg(concat_pred_outsample)

  metrics = [ATE_abs_insample.numpy(), ATE_TARREG_insample.numpy(), ITE_RMSE_insample.numpy(), ATE_TARREG_outsample.numpy(), PEHE_outsample.numpy(), PEHE_tareg_outsample.numpy(), elapsed_time_minutes]
  sim_evals.append(metrics)

10
/content/drive/MyDrive/Causal_network_matching/data/BlogCatalog2/BlogCatalog9.mat
[[0. 1. 1. ... 0. 0. 0.]
 [1. 1. 1. ... 0. 0. 0.]
 [2. 1. 1. ... 0. 0. 0.]
 ...
 [4. 1. 1. ... 0. 0. 0.]
 [2. 2. 1. ... 0. 0. 0.]
 [1. 1. 1. ... 0. 0. 0.]]
Epoch 1/100000
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step - loss: 7600.6553 - regression_loss: 3543.3206 - tarreg_loss: 7595.9492 - treatment_acc: 0.5105 - val_loss: 2312.2354 - val_regression_loss: 1156.2117 - val_tarreg_loss: 2307.5293 - val_treatment_acc: 0.5216 - learning_rate: 6.0000e-08
Epoch 2/100000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step - loss: 6997.5508 - regression_loss: 3516.7434 - tarreg_loss: 6992.8447 - treatment_acc: 0.5096 - val_loss: 2298.6694 - val_regression_loss: 1151.9342 - val_tarreg_loss: 2293.9634 - val_treatment_acc: 0.5216 - learning_rate: 6.0000e-08
Epoch 3/100000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step - loss: 7032.2393 - regression_loss: 3483.0771 - tarreg_loss: 7027.5337 - treatment_acc: 0.5090 - val_loss: 2289.5571 - val_regres

In [20]:
sim_evals = np.asarray(sim_evals)
# with open('eval_metrics_news_20.npy', 'wb') as f:
#     np.save(f,sim_evals)

In [21]:
# [ATE_abs_insample.numpy(), ATE_TARREG_insample.numpy(), ITE_RMSE_insample.numpy(), ATE_TARREG_outsample.numpy(), PEHE_outsample.numpy(), PEHE_tareg_outsample.numpy(), elapsed_time_minutes

print(" Insample ATE error for Dragonnet(mean over 10) =", round(np.mean(sim_evals[:,0]),2),"+-",round((np.std(sim_evals[:,0], ddof=1) / np.sqrt(np.size(sim_evals[:,0]))),2))
print(" Insample ATE targ_reg error for Dragonnet(mean over 10) =", round(np.mean(sim_evals[:,1]),2),"+-",round((np.std(sim_evals[:,1], ddof=1) / np.sqrt(np.size(sim_evals[:,1]))),2))
print(" Insample ITE_RMSE error for Dragonnet(mean over 10) =", round(np.mean(sim_evals[:,2]),2),"+-",round((np.std(sim_evals[:,2], ddof=1) / np.sqrt(np.size(sim_evals[:,2]))),2))
print(" Outsample ATE targ_reg error for Dragonnet(mean over 10) =", round(np.mean(sim_evals[:,3]),2),"+-",round((np.std(sim_evals[:,3], ddof=1) / np.sqrt(np.size(sim_evals[:,3]))),2))
print(" Outsample PEHE error for Dragonnet(mean over 10) =", round(np.mean(sim_evals[:,4]),2),"+-",round((np.std(sim_evals[:,4], ddof=1) / np.sqrt(np.size(sim_evals[:,4]))),2))
print(" Outsample PEHE targ_reg error for Dragonnet(mean over 10) =", round(np.mean(sim_evals[:,5]),2),"+-",round((np.std(sim_evals[:,5], ddof=1) / np.sqrt(np.size(sim_evals[:,5]))),2))
print(" Wall time for TARNET(sum over 10) =", np.sum(sim_evals[:,6]))

 Insample ATE error for Dragonnet(mean over 10) = 4.21 +- 1.03
 Insample ATE targ_reg error for Dragonnet(mean over 10) = 3.94 +- 0.96
 Insample ITE_RMSE error for Dragonnet(mean over 10) = 24.05 +- 0.37
 Outsample ATE targ_reg error for Dragonnet(mean over 10) = 3.81 +- 1.0
 Outsample PEHE error for Dragonnet(mean over 10) = 23.03 +- 2.58
 Outsample PEHE targ_reg error for Dragonnet(mean over 10) = 23.16 +- 2.6
 Wall time for TARNET(sum over 10) = 9.218615098794302


In [18]:
from tensorflow.python.client import device_lib

device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 13723987066099933871
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 40427651072
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 2617528058496593902
 physical_device_desc: "device: 0, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:00:04.0, compute capability: 8.0"
 xla_global_id: 416903419]